In [1]:
# To find out where the pyspark
import findspark
findspark.init()

In [2]:
# Creating Spark Context
from pyspark import SparkContext, SparkConf

conf = (SparkConf()
         .setMaster("local")
         .setAppName("pbl-2")
         .set("spark.executor.cores", "4")) # 코어 수

sc = SparkContext(conf = conf)

In [11]:
# nn = 10

# def f(pdata, k):
#     for x in pdata:
#         yield (x[0], x[1]+k)

# x = sc.parallelize([('1',1), ('2',2), ('1',3), ('2',4)],4)
# y = x.partitionBy(2, lambda x: int(x[0])).mapPartitions(lambda x: f(x, k=5))
# print(y.collect())

[('2', 7), ('2', 9), ('1', 6), ('1', 8)]


In [4]:
import matplotlib.pyplot as plt
import numpy as np
import math
import time

#### 데이터셋 생성 및 변수 정의

(Source, Destination set) 맵핑

In [5]:
data = sc.textFile("test_dataset.txt").flatMap(lambda file: file.split('\n')).map(lambda line: line.split(' '))\
        .map(lambda line: (int(line[0]), [int(i) for i in line[1:]]))

N = data.count()
block = [16, 8, 4, 2, 1]
maxIter = 20
execution_time = []

print(data.take(1))

[(0, [137, 781, 1])]


In [13]:
def pr_red(partition_data, block_size, N):
    new_pr = [0.15 * 1/N for _ in range(block_size)]
    partition_num = int(partition_data[0][0]) // block_size
    
    for element in partition_data:
        dest = int(element[0])
        new_pr[dest % block_size] += element[1]
        
    for i in range(block_size):
        yield new_pr[i]

#### Block Based Page Rank Calculation

1. destination node id의 범위에 따라 블록 개수만큼 파티션을 나눠준다.
  
  
2. 각 파티션별로 Page Rank를 계산한다.
  
  
3. 모든 파티션의 Page Rank를 합친다.

In [14]:
for block_cnt in block: # block의 개수를 바꿔가며 실행
    start = time.time()
    
    pr = sc.broadcast([ 1. / float(N) for _ in range(N) ]) 
    block_size = (N // block_cnt) if N % block_cnt != 0 else (N // block_cnt)+1
        
    for _ in range(maxIter):
        itr = data.flatMap(lambda s: [(str(dest), pr.value[s[0]] / len(s[1])) for dest in s[1]])\
                    .partitionBy(block_cnt, lambda x: int(x[0]) // block_size).mapPartitions(lambda x: pr_red(x, block_size, N))
        pr = sc.broadcast(itr.collect())
        
    end = time.time()
    execution_time.append(end-start)

    print("the number of blocks: {}".format(block_cnt))
    print("duration: {}".format(end-start))
    print()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 13.0 failed 1 times, most recent failure: Lost task 1.0 in stage 13.0 (TID 27, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/pyspark/pkg/spark-2.4.5-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/home/pyspark/pkg/spark-2.4.5-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/pyspark/pkg/spark-2.4.5-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 400, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "<ipython-input-13-e073adaab0b8>", line 3, in pr_red
TypeError: 'itertools.chain' object is not subscriptable

	at org.apache.spark.ap

#### 그래프

In [ ]:
plt.plot(block, execution_time, marker = 'o')

plt.xlabel("Number of blocks")
plt.ylabel("Execution time")

plt.yscale('log')
plt.show()